In [3]:
import pandas as pd
import ast
import numpy as np
df=pd.read_csv("frac_credits.csv")

# Desanidado de columnas 

## Comenzamos con la columna Cast

In [4]:
#Relleno los Nulos y separo la Columna Cast

data=df["cast"]

data=data.str.replace("[]"," 'cast_id': null, 'character': null, 'credit_id': null, 'gender': null, 'id': null, 'name': null, 'order': null, 'profile_path': null'")

In [5]:
#Reemplazamos argumentos para poder separar y extraer las variabble que necesitamos

data=data.str.replace("{"," ")
data=data.str.replace("[","")
data=data.str.replace("]","")
data=data.str.replace("'cast_id':","")
data=data.str.replace("'character':","")
data=data.str.replace("'credit_id':","")
data=data.str.replace("'gender':","")
data=data.str.replace("'id':","")
data=data.str.replace("'name':","")
data=data.str.replace("'order':","")
data=data.str.replace("'profile_path':","")


In [6]:
#Creamos una funcion para poder separar los diccionarios de las filas

def Separar(lista):
    dat=lista.split("} ,")
    dat=pd.DataFrame(dat)
    dat=dat
   
    return dat

#Creamos funcion para separa en columnas cada variable

def fila(lista):
    dat=lista.split(", ")
    dat=pd.DataFrame(dat)
    dat=dat.T
   
    return dat

In [7]:
#Funcion por la cual vamos a hacer la Limpieza y quedarnos solamente con los actores

def Cast(x):
    var=x
    cast=Separar(var)
    cast=cast[0]
    cast=cast.str.replace("} ","")
    cast=pd.concat(list(map(lambda i: fila(i), cast)))

    cast=cast[[5]]
    cast.reset_index(inplace=True)
    cast=cast.rename(columns={5:"Actors"})
    cast=cast.drop(["index"],axis=1)
    cast["Actors"]=cast["Actors"].astype(str)

    cast=cast.T
    cast=cast.apply(lambda x: ''.join(x.astype(str)), axis=1)
    cast=pd.DataFrame(cast)

    return cast

In [8]:
cast=pd.DataFrame()
cast=pd.concat(list(map(lambda i: Cast(i), data)))


In [9]:
cast=cast.reset_index()
cast=cast.drop("index",axis=1)
cast=cast.rename(columns={0:"Actor"})
cast=cast["Actor"]
cast=pd.DataFrame(cast)
cast.head(3)

,Actor
0,'Tom Hanks'
1,'Robin Williams'
2,'Walter Matthau'


## Desanidado de la columna Crew

In [10]:
data=df["crew"]
data=data.str.replace("[]"," \'credit_id\': null ,\'department\': null,\'gender\': null, \'id\': null,\'job\': null,\'name\': null, \'profile_path\': null")

In [11]:
data=data.str.replace("{","")
data=data.str.replace("[","")
data=data.str.replace("]","")
data=data.str.replace("\'credit_id\':","")
data=data.str.replace("\'department\':","")
data=data.str.replace("\'gender\':","")
data=data.str.replace("\'name\':","")
data=data.str.replace(" \'id\':","")
data=data.str.replace("\'name\':","")
data=data.str.replace(" \'profile_path\': ","")


In [12]:
def Separar_Crew(lista):
    dat=lista.split("},")
    dat=pd.DataFrame(dat)
    dat=dat
   
    return dat

def fila_Crew(lista):
    dat=lista.split(",")
    dat=pd.DataFrame(dat)
    dat=dat.T
   
    return dat

In [13]:
def Director(x):
    direc=Separar_Crew(x)
    direc=direc[0]
    direc=direc.str.replace("}","")

    crew= pd.concat(list(map(lambda i: fila_Crew(i), direc)))

    crew= crew[[4,5]]
    crew.reset_index(inplace=True)
    crew=crew.rename(columns={4:"job"})
    crew=crew.drop(["index"],axis=1)
    crew["job"]=crew["job"].astype(str)
    crew.head()

    resultado=crew.loc[crew["job"]==" 'job': 'Director'"]
    resultado=resultado[5]
    resultado=pd.DataFrame(resultado)
    resultado=resultado.T
    resultado["Director"] =resultado.apply(lambda x: ''.join(x.astype(str)), axis=1)
    resultado=resultado["Director"]
    resultado=pd.DataFrame(resultado)
    
    return resultado

In [14]:
Direc=pd.DataFrame()
Direc=pd.concat(list(map(lambda i: Director(i), data)))


In [15]:
Direc=Direc.reset_index()
Direc=Direc.drop("index",axis=1)
Direc=Direc["Director"]
Direc=pd.DataFrame(Direc)


## Una ves desanidada las columnas las concatenamos para que quede un archivo para seguir trabajando con el mismo

In [16]:
credit=pd.concat([df,cast,Direc],axis=1)
credit=credit[["id","Actor","Director"]]


In [17]:
credit

,id,Actor,Director
0,862,'Tom Hanks','John Lasseter'
1,8844,'Robin Williams','Joe Johnston'
2,15602,'Walter Matthau','Howard Deutch'
3,31357,'Whitney Houston','Forest Whitaker'
4,11862,'Steve Martin','Charles Shyer'
...,...,...,...
45471,439050,'Leila Hatami','Hamid Nematollah'
45472,111109,'Angel Aquino','Lav Diaz'
45473,67758,'Erika Eleniak','Mark L. Lester'
45474,227506,'Iwan Mosschuchin','Yakov Protazanov'


In [18]:
credit.to_csv("../03 - Armado el Completo/Credit_desani.csv",index=False)